# SETTING UP CONNECTION TO SQL SERVER WITH DATASETS

In [2]:
import pandas as pd
import sqlite3
from IPython.display import display

conn = sqlite3.connect("pharm_data.db")

#Fetching list of tables we have access to, for reference
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ALL TABLES: ",)
for table in tables: print(table[0],end=", ")
print("\b\b")

ALL TABLES: 
BENEFICIARY_COST, GEOGRAPHIC_LOCATOR, PLAN_INFO, ZIP_STATE_COUNTY, NCPDP, FORMULARY_DATA, NDC_DATA, RXCUI_DATA, PNF_DATA, ALL_DATA, 


# REQUESTING USER'S ZIP TO COMPUTE PDP_REGION_CODE

In [3]:
#Asks user for their zip code, gets state, determines pdp_region_code based on state
user_zip = input("Enter your zipcode: ")
user_state = pd.read_sql("select COUNTY,STATENAME\
                          from ZIP_STATE_COUNTY\
                          where ZIPCODE is '" + user_zip + "'", conn)\
            ['STATENAME'].values[0].rstrip()

user_pdp_region_code = pd.read_sql("select distinct PDP_REGION_CODE\
                                    from GEOGRAPHIC_LOCATOR\
                                    where STATENAME is '"+user_state+"'",conn)\
                       ['PDP_REGION_CODE'].values[0]

Enter your zipcode: 07040


# REQUESTING DRUGS FROM USER, YIELDING RXCUIs

In [4]:
user_generics = {(input("Enter a generic drug name: ").upper(),\
                  input("Enter a desired strength: ")):None\
                 for i in range(int(input("Enter how many drugs you wish to search: ")))}
for drug,strength in user_generics:
    user_generics[(drug,strength)] = pd.read_sql("select * from RXCUI_DATA where upper(PSN) like '"+drug+"%' and strength like '"+strength+" %MG' and GENERIC_RXCUI is ''",conn).drop_duplicates(subset='SXDG_RXCUI',keep='last')['RXCUI'].values[0]
print(user_generics)

Enter how many drugs you wish to search: 3
Enter a generic drug name: metoprolol succinate
Enter a desired strength: 50
Enter a generic drug name: allopurinol
Enter a desired strength: 300
Enter a generic drug name: metformin
Enter a desired strength: 500
{('METOPROLOL SUCCINATE', '50'): '866436', ('ALLOPURINOL', '300'): '197320', ('METFORMIN', '500'): '861007'}


# GETTING DATA FOR FORMULARY_IDs IN REGION

In [5]:
out = pd.read_sql("select \
                    PLAN_INFO.CONTRACT_NAME, \
                    PLAN_INFO.PLAN_NAME, \
                    PLAN_INFO.CONTRACT_ID, \
                    PLAN_INFO.PLAN_ID, \
                    PLAN_INFO.SEGMENT_ID, \
                    PLAN_INFO.FORMULARY_ID, \
                    PLAN_INFO.PREMIUM, \
                    PLAN_INFO.DEDUCTIBLE \
            from \
                FORMULARY_DATA join PLAN_INFO \
                on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
            where \
                  PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
                  PLAN_INFO.CONTRACT_ID like 'S%'\
            order by PLAN_INFO.FORMULARY_ID",conn)\
.drop_duplicates(subset='FORMULARY_ID',keep='first')\
.reset_index(drop=True)
display(out)

,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE
0,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0
1,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Saver Plus (PDP),S5921,349,0,18040,33.2,405.0
2,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0
3,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0
4,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0
5,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0
6,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0
7,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure (PDP),S5617,18,0,18082,74.8,405.0
8,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0
9,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Standard (PDP),S5993,1,0,18130,47.7,405.0


# YIELDING PLAN SELECTION OUTPUT TABLE
###### EXTREMELY FAST NOW

1. CONSTRUCTING INITIAL TABLE
2. PUTTING EACH DRUG IN THE TABLE, COMPUTING COST
3. SHOWING TOTAL COST FOR EACH ITERATION

In [6]:
out = pd.read_sql("select \
                    PLAN_INFO.CONTRACT_NAME, \
                    PLAN_INFO.PLAN_NAME, \
                    PLAN_INFO.CONTRACT_ID, \
                    PLAN_INFO.PLAN_ID, \
                    PLAN_INFO.SEGMENT_ID, \
                    PLAN_INFO.FORMULARY_ID, \
                    PLAN_INFO.PREMIUM, \
                    PLAN_INFO.DEDUCTIBLE \
            from \
                FORMULARY_DATA join PLAN_INFO \
                on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
            where \
                  PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
                  PLAN_INFO.CONTRACT_ID like 'S%'\
            order by PLAN_INFO.FORMULARY_ID",conn)\
.drop_duplicates(subset='FORMULARY_ID',keep='first')\
.reset_index(drop=True)

for key in user_generics:
    out = pd.concat([out,
               pd.read_sql(" ".join(("select \
                        PLAN_INFO.CONTRACT_NAME, \
                        PLAN_INFO.PLAN_NAME, \
                        PLAN_INFO.CONTRACT_ID, \
                        PLAN_INFO.PLAN_ID, \
                        PLAN_INFO.SEGMENT_ID, \
                        PLAN_INFO.FORMULARY_ID, \
                        PLAN_INFO.PREMIUM, \
                        PLAN_INFO.DEDUCTIBLE, \
                        FORMULARY_DATA.RXCUI as "+key[0].replace(" ","_")+"_RXCUI, \
                        FORMULARY_DATA.PRIOR_AUTHORIZATION_YN as "+key[0].replace(" ","_")+"_PA, \
                        FORMULARY_DATA.TIER_LEVEL_VALUE as "+key[0].replace(" ","_")+"_TIER, \
                        BENEFICIARY_COST.COST_AMT_PREF as "+key[0].replace(" ","_")+"_CAP \
                from \
                    FORMULARY_DATA join PLAN_INFO \
                        on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
                    join BENEFICIARY_COST \
                        on PLAN_INFO.CONTRACT_ID = BENEFICIARY_COST.CONTRACT_ID and \
                           PLAN_INFO.PLAN_ID = BENEFICIARY_COST.PLAN_ID and \
                           cast(FORMULARY_DATA.TIER_LEVEL_VALUE as integer) = \
                                                    cast(BENEFICIARY_COST.TIER as integer) \
                where \
                      PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
                      FORMULARY_DATA.RXCUI is "+user_generics[key]+" and \
                      PLAN_INFO.CONTRACT_ID like 'S%' and \
                      (cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 0 or \
                      cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 1) and \
                      BENEFICIARY_COST.DAYS_SUPPLY = 1 and \
                      cast(BENEFICIARY_COST.COST_TYPE_PREF as integer) = 1\
                order by PLAN_INFO.FORMULARY_ID").split()),conn)\
    .drop_duplicates(subset='FORMULARY_ID',keep='first')\
    .reset_index(drop=True)\
    [[key[0].replace(" ","_")+'_RXCUI',
      key[0].replace(" ","_")+'_PA',
      key[0].replace(" ","_")+'_TIER',
      key[0].replace(" ","_")+'_CAP']]],axis=1)
    #ADD IN COST COLUMN
    if 'ANNUAL_TOTAL_COST' in out.columns:
        del out['ANNUAL_TOTAL_COST']
    if 'MONTHLY_COPAY' in out.columns:
        del out['MONTHLY_COPAY']
    out['ANNUAL_TOTAL_COST'] = (out[[c for c in out.columns if c[-3:] == 'CAP']]*12).sum(axis=1) + out['PREMIUM'] + out['DEDUCTIBLE']
    out['MONTHLY_COPAY'] = out[[c for c in out.columns if c[-3:] == 'CAP']].sum(axis=1)
    out = out.sort_values(by='ANNUAL_TOTAL_COST').reset_index(drop=True)
    display(out)



,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,METOPROLOL_SUCCINATE_RXCUI,METOPROLOL_SUCCINATE_PA,METOPROLOL_SUCCINATE_TIER,METOPROLOL_SUCCINATE_CAP,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,866436,N,2,2.0,80.2,2.0
1,SILVERSCRIPT INSURANCE COMPANY,SilverScript Plus (PDP),S5601,9,0,18420,84.6,0.0,866436,N,1,1.0,96.6,1.0
2,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,866436,N,1,4.0,100.9,4.0
3,HUMANA INSURANCE COMPANY,Humana Enhanced (PDP),S5884,62,0,18256,77.8,0.0,866436,N,2,7.0,161.8,7.0
4,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Enhanced (PDP),S5993,3,0,18131,99.6,0.0,866436,N,2,8.0,195.6,8.0
5,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,8,0,18419,34.3,0.0,866436,N,2,14.0,202.3,14.0
6,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0,866436,N,2,10.0,206.6,10.0
7,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,866436,N,2,2.0,383.9,2.0
8,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0,866436,N,1,0.0,431.8,0.0
9,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0,866436,N,1,1.0,447.4,1.0


,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,METOPROLOL_SUCCINATE_RXCUI,METOPROLOL_SUCCINATE_PA,METOPROLOL_SUCCINATE_TIER,METOPROLOL_SUCCINATE_CAP,ALLOPURINOL_RXCUI,ALLOPURINOL_PA,ALLOPURINOL_TIER,ALLOPURINOL_CAP,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,SILVERSCRIPT INSURANCE COMPANY,SilverScript Plus (PDP),S5601,9,0,18420,84.6,0.0,866436,N,1,1.0,197320,N,1,1.0,108.6,2.0
1,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,866436,N,1,4.0,197320,N,1,1.0,112.9,5.0
2,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,866436,N,2,2.0,197320,N,1,6.0,152.2,8.0
3,HUMANA INSURANCE COMPANY,Humana Enhanced (PDP),S5884,62,0,18256,77.8,0.0,866436,N,2,7.0,197320,N,1,0.0,161.8,7.0
4,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,8,0,18419,34.3,0.0,866436,N,2,14.0,197320,N,1,0.0,202.3,14.0
5,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Enhanced (PDP),S5993,3,0,18131,99.6,0.0,866436,N,2,8.0,197320,N,1,1.0,207.6,9.0
6,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0,866436,N,2,10.0,197320,N,1,1.0,218.6,11.0
7,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,866436,N,2,2.0,197320,N,1,1.0,395.9,3.0
8,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Saver Plus (PDP),S5921,349,0,18040,33.2,405.0,866436,N,1,1.0,197320,N,1,1.0,462.2,2.0
9,"WELLCARE PRESCRIPTION INSURANCE, INC.",WellCare Classic (PDP),S4802,78,0,18443,36.9,405.0,866436,N,2,1.0,197320,N,1,1.0,465.9,2.0


,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,METOPROLOL_SUCCINATE_RXCUI,METOPROLOL_SUCCINATE_PA,...,ALLOPURINOL_RXCUI,ALLOPURINOL_PA,ALLOPURINOL_TIER,ALLOPURINOL_CAP,METFORMIN_RXCUI,METFORMIN_PA,METFORMIN_TIER,METFORMIN_CAP,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,866436,N,...,197320,N,1,1.0,861007,N,1,1.0,124.9,6.0
1,HUMANA INSURANCE COMPANY,Humana Enhanced (PDP),S5884,62,0,18256,77.8,0.0,866436,N,...,197320,N,1,0.0,861007,N,1,0.0,161.8,7.0
2,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,866436,N,...,197320,N,1,6.0,861007,N,1,1.0,164.2,9.0
3,SILVERSCRIPT INSURANCE COMPANY,SilverScript Plus (PDP),S5601,9,0,18420,84.6,0.0,866436,N,...,197320,N,1,1.0,861007,N,1,6.0,180.6,8.0
4,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Enhanced (PDP),S5993,3,0,18131,99.6,0.0,866436,N,...,197320,N,1,1.0,861007,N,1,0.0,207.6,9.0
5,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,8,0,18419,34.3,0.0,866436,N,...,197320,N,1,0.0,861007,N,1,1.0,214.3,15.0
6,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0,866436,N,...,197320,N,1,1.0,861007,N,1,1.0,230.6,12.0
7,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,866436,N,...,197320,N,1,1.0,861007,N,1,1.0,407.9,4.0
8,HUMANA INSURANCE COMPANY,Humana Preferred Rx Plan (PDP),S5884,131,0,18254,37.0,405.0,866436,N,...,197320,N,1,1.0,861007,N,1,1.0,478.0,3.0
9,HUMANA INSURANCE COMPANY,Humana Walmart Rx Plan (PDP),S5884,150,0,18255,20.4,405.0,866436,N,...,197320,N,1,0.0,861007,N,1,1.0,485.4,5.0


# FULL FINAL OUTPUT, SORTED BY ANNUAL TOTAL COST

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 22):
    display(out)

,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,METOPROLOL_SUCCINATE_RXCUI,METOPROLOL_SUCCINATE_PA,METOPROLOL_SUCCINATE_TIER,METOPROLOL_SUCCINATE_CAP,ALLOPURINOL_RXCUI,ALLOPURINOL_PA,ALLOPURINOL_TIER,ALLOPURINOL_CAP,METFORMIN_RXCUI,METFORMIN_PA,METFORMIN_TIER,METFORMIN_CAP,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,866436,N,1,4.0,197320,N,1,1.0,861007,N,1,1.0,124.9,6.0
1,HUMANA INSURANCE COMPANY,Humana Enhanced (PDP),S5884,62,0,18256,77.8,0.0,866436,N,2,7.0,197320,N,1,0.0,861007,N,1,0.0,161.8,7.0
2,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,866436,N,2,2.0,197320,N,1,6.0,861007,N,1,1.0,164.2,9.0
3,SILVERSCRIPT INSURANCE COMPANY,SilverScript Plus (PDP),S5601,9,0,18420,84.6,0.0,866436,N,1,1.0,197320,N,1,1.0,861007,N,1,6.0,180.6,8.0
4,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Enhanced (PDP),S5993,3,0,18131,99.6,0.0,866436,N,2,8.0,197320,N,1,1.0,861007,N,1,0.0,207.6,9.0
5,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,8,0,18419,34.3,0.0,866436,N,2,14.0,197320,N,1,0.0,861007,N,1,1.0,214.3,15.0
6,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0,866436,N,2,10.0,197320,N,1,1.0,861007,N,1,1.0,230.6,12.0
7,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,866436,N,2,2.0,197320,N,1,1.0,861007,N,1,1.0,407.9,4.0
8,HUMANA INSURANCE COMPANY,Humana Preferred Rx Plan (PDP),S5884,131,0,18254,37.0,405.0,866436,N,2,1.0,197320,N,1,1.0,861007,N,1,1.0,478.0,3.0
9,HUMANA INSURANCE COMPANY,Humana Walmart Rx Plan (PDP),S5884,150,0,18255,20.4,405.0,866436,N,2,4.0,197320,N,1,0.0,861007,N,1,1.0,485.4,5.0


# FINDING LOCATIONS IN ZIP THAT OFFER CHEAPEST PLAN
##### TAKES A LONGER TIME BECAUSE PNF_DATA IS HUGE

In [9]:
pd.read_sql("select NCPDP.PHARMACY_NAME, NCPDP.PHONE, PNF_DATA.* \
             from NCPDP, PNF_DATA\
             where PNF_DATA.CONTRACT_ID='"+out.iloc[0].CONTRACT_ID+"' and \
                   cast(PNF_DATA.PLAN_ID as integer)="+str(out.iloc[0].PLAN_ID)+" and \
                   cast(PNF_DATA.SEGMENT_ID as integer)="+str(out.iloc[0].SEGMENT_ID)+" and \
                   PNF_DATA.PREFERRED_STATUS_RETAIL='Y' and \
                   PNF_DATA.PHARMACY_ZIPCODE like '"+user_zip[:-1]+"%' and \
                   NCPDP.NPI is substr(PNF_DATA.PHARMACY_NUMBER,3) \
             limit 10",conn)

,PHARMACY_NAME,PHONE,CONTRACT_ID,PLAN_ID,SEGMENT_ID,PHARMACY_NUMBER,PHARMACY_ZIPCODE,PREFERRED_STATUS_RETAIL,PREFERRED_STATUS_MAIL,PHARMACY_RETAIL,PHARMACY_MAIL,IN_AREA_FLAG,BRAND_DISPENSING_FEE_30,BRAND_DISPENSING_FEE_60,BRAND_DISPENSING_FEE_90,GENERIC_DISPENSING_FEE_30,GENERIC_DISPENSING_FEE_60,GENERIC_DISPENSING_FEE_90
0,TERRYS FAMILY PHARMACY INC,9732392200,S5617,249,000,101053405464,07044,Y,N,Y,N,1,0.75,0.75,0.75,0.75,0.4,0.4
1,KEILS PHARMACY,9737442113,S5617,249,000,101104862564,07043,Y,N,Y,N,1,0.75,0.75,0.75,0.75,0.4,0.4
2,NORTH BERGEN PHARMACY,2018686900,S5617,249,000,101992713283,07047,Y,N,Y,N,1,0.5,0.5,0.5,0.6,0.6,0.6
3,TOWN AND COUNTRY PHARMACY,9737622700,S5617,249,000,101194716548,07040,Y,N,Y,N,1,0.5,0.5,0.5,0.6,0.6,0.6
4,ATLAS DRUG AND NUTRITION,2018695990,S5617,249,000,101417992629,07047,Y,N,Y,N,1,0.5,0.5,0.5,0.6,0.6,0.6
5,SHOPRITE PHARMACY OF COLUMBIA PARK,2018657050,S5617,249,000,101992810774,07047,Y,N,Y,N,1,0.4,0,0,0.4,0,0
6,LEXANN PHARMACY & WELLNESS CENTER,9737630010,S5617,249,000,101780082339,07040,Y,N,Y,N,1,0.75,0.75,0.75,0.75,0.4,0.4
7,ACME PHARMACY #1764,9737467471,S5617,249,000,101457728271,07043,Y,N,Y,N,1,0.3,0,0,0.55,0,0
8,AMEEN PHARMACY,2015908776,S5617,249,000,101619114170,07047,Y,N,Y,N,1,0.5,0.5,0.5,0.6,0.6,0.6
9,WALGREENS #13701,9735590901,S5617,249,000,101780826776,07044,Y,N,Y,N,1,1.3,0,0,1.3,0,0
